<a href="https://colab.research.google.com/github/donal0c/HuggingFaceTutorials/blob/main/Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [4]:
from openai import OpenAI
client = OpenAI(api_key="sk-**************")

In [5]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)

In [7]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_XcuD4m4Ck6tYh3W7fqE1wPg2', created_at=1699898134, metadata={}, object='thread')


In [10]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "Solve this problem: 3x + 11 = 14"
)

In [11]:
print(message)

ThreadMessage(id='msg_JvfVPNmHOqoNHaEzoOz5uYbL', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Solve this problem: 3x + 11 = 14'), type='text')], created_at=1699898276, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_XcuD4m4Ck6tYh3W7fqE1wPg2')


In [12]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

In [13]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

In [14]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [15]:
for message in reversed(messages.data):
  print(message.role + ": " + message.content[0].text.value)

user: Solve this problem: 3x + 11 = 14
assistant: The solution to the equation \(3x + 11 = 14\) is \(x = 1\).


In [16]:
file = client.files.create(
    file=open("/content/memgpt-paper.pdf", "rb"),
    purpose = "assistants"
)

In [17]:
print(file)

FileObject(id='file-AHdLsEmGS3AtxF7JPXAi3HOL', bytes=602501, created_at=1699902360, filename='memgpt-paper.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [18]:
assistant = client.beta.assistants.create(
    name="Machine Learning researcher",
    instructions="You are a machine learning researcher, answer questions on the resuarch paper ",
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids = ['file-AHdLsEmGS3AtxF7JPXAi3HOL']
)

In [20]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_Ha4SEs4KiUdffHVS0aN8B0UD', created_at=1699902590, metadata={}, object='thread')


In [23]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "How does memgpt allow LLMS to have unlimited context length"
)

In [24]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

In [25]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

In [27]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)


In [28]:
for message in reversed(messages.data):
  print(message.role + ": " + message.content[0].text.value)

user: How does memgpt allow LLMS to have unlimited context length
user: How does memgpt allow LLMS to have unlimited context length
user: How does memgpt allow LLMS to have unlimited context length
assistant: MemGPT allows LLMS (Large Language Model Systems) to handle what effectively seems like unlimited context length by utilizing several strategies for memory management:

1. **Main Context Management**: The main context within MemGPT is divided into three components: system instructions (read-only and pinned), conversational context (read-only with a special eviction policy), and working context (writeable by the LLM processor). These combined cannot exceed the maximum context size of the underlying LLM processor. When the conversational context reaches a certain size, a portion of it is either truncated or compressed via recursive summarization【11†source】.

2. **External Context**: MemGPT uses out-of-context storage, referred to as 'external context', which functions similarly to d